In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment3/'
FOLDERNAME = 'comp451/Project/visual-transformer'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
import os
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))

ROOT_PATH = PATH = '/content/drive/MyDrive/comp451/Project/visual-transformer'

os.chdir(ROOT_PATH)


Mounted at /content/drive


In [2]:
%load_ext tensorboard

In [3]:
import torch
import torch.nn.functional as F
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, Subset
from torchvision import models
from models.vt_resnet import VTResNet
from models.resnet import BasicBlock
import matplotlib.pyplot as plt
import random
import pickle
import PIL
import numpy as np

In [4]:
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

In [5]:
def random_seed():
    np.random.seed(8)
    torch.manual_seed(8)
    random.seed(8)

In [6]:
!rm -rf ./logs/

In [7]:
def get_data(name):

    transform_train = torchvision.transforms.Compose(
     [
        # torchvision.transforms.RandomHorizontalFlip(),
        # torchvision.transforms.RandomRotation(10, resample=PIL.Image.BILINEAR),
        # torchvision.transforms.RandomAffine(8, translate=(.15,.15)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Resize((32,32)),
        torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)

    transform_valid = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Resize((32,32)),
        torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    if name == 'CIFAR10':
        data_train = torchvision.datasets.CIFAR10('/data', train=True, download=True, transform=transform_train)
        data_valid = torchvision.datasets.CIFAR10('/data', train=False, download=True, transform=transform_valid)

        return data_train, data_valid
    
    if name == 'MNIST':
        data_train = torchvision.datasets.MNIST('/data', train=True, download=True, transform=transform_train)
        data_valid = torchvision.datasets.MNIST('/data', train=False, download=True, transform=transform_valid)

        return data_train, data_valid

    if name == 'ImageNet':
        data_train = torchvision.datasets.ImageNet('/content/drive/MyDrive/comp451/Project/visual-transformer/data', split='train', transform=transform_train)
        data_valid = torchvision.datasets.ImageNet('/content/drive/MyDrive/comp451/Project/visual-transformer/data', split='val', transform=transform_valid)

        return data_train, data_valid

In [8]:
data_train, data_valid = get_data('CIFAR10')

Files already downloaded and verified
Files already downloaded and verified


In [9]:
train_loader = DataLoader(data_train, batch_size=4, shuffle=True)
valid_loader = DataLoader(data_valid, batch_size=4, shuffle=True)

In [10]:
L = 8
D = 512
layer_plane = [64, 128, 256, 512]
enc_layer = 2
n_head = 8
fc_dim = 1024
dropout = 0.5
model = VTResNet(
    resnet_block=BasicBlock,
    layers=[2,2,2,2],
    tokens=L,
    token_channels=D,
    input_dim=32,
    layer_planes=layer_plane,
    transformer_enc_layers=enc_layer,
    transformer_heads=n_head,
    transformer_fc_dim=fc_dim,
    transformer_dropout=dropout,
    num_classes=10,
    image_channels=3
)
 
model.load_state_dict(torch.load(ROOT_PATH + '/saved_models/model1.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [11]:
print(model.vt_layers[0].tokenizer.linear1.weight)

Parameter containing:
tensor([[-0.0041,  0.0577,  0.0451,  ...,  0.0391, -0.0235,  0.0219],
        [-0.0603, -0.0192,  0.2200,  ...,  0.0099,  0.0763,  0.0357],
        [ 0.0048,  0.0481,  0.0566,  ..., -0.1940,  0.0467, -0.0719],
        ...,
        [-0.0710, -0.0377, -0.0459,  ..., -0.0771, -0.0134,  0.0165],
        [ 0.0152,  0.0658,  0.0200,  ...,  0.0690, -0.0393,  0.0198],
        [ 0.1962,  0.0982, -0.0028,  ...,  0.1017,  0.0297, -0.1728]],
       requires_grad=True)


In [12]:
writer = SummaryWriter('logs')

In [13]:
writer.add_histogram('vt_layer_1_tokenizer_linear1_weights', model.vt_layers[1].tokenizer.linear1.weight, global_step=1)
writer.add_histogram('vt_layer_0_tokenizer_linear1_weights', model.vt_layers[0].tokenizer.linear1.weight, global_step=1)

In [14]:
classes = ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J')

In [15]:
# def select_n_random(data, labels, n=100):
#     '''
#     Selects n random datapoints and their corresponding labels from a dataset
#     '''
#     assert len(data) == len(labels)

#     perm = torch.randperm(len(data))
#     return data[perm][:n], labels[perm][:n]

# # select random images and their target indices
# images, labels = select_n_random(data_train.data, np.array(data_train.targets))

# # get the class labels for each image
# class_labels = [classes[lab] for lab in labels]

# # log embeddings
# features = images.reshape(-1, 32 * 32 * 3)
# writer.add_embedding(features,
#                     metadata=class_labels)
# writer.close()

In [ ]:
# 1. gets the probability predictions in a test_size x num_classes Tensor
# 2. gets the preds in a test_size Tensor
# takes ~10 seconds to run
class_probs = []
class_preds = []
with torch.no_grad():
    for data in valid_loader:
        images, labels = data
        output = model(images)
        class_probs_batch = [F.softmax(el, dim=0) for el in output]
        _, class_preds_batch = torch.max(output, 1)

        class_probs.append(class_probs_batch)
        class_preds.append(class_preds_batch)

test_probs = torch.cat([torch.stack(batch) for batch in class_probs])
test_preds = torch.cat(class_preds)

def add_pr_curve_tensorboard(class_index, test_probs, test_preds, global_step=0):
    '''
    Takes in a "class_index" from 0 to 9 and plots the corresponding
    precision-recall curve
    '''
    tensorboard_preds = test_preds == class_index
    tensorboard_probs = test_probs[:, class_index]

    writer.add_pr_curve(classes[class_index],
                        tensorboard_preds,
                        tensorboard_probs,
                        global_step=global_step)
    # writer.close()

# plot all the pr curves
for i in range(len(classes)):
    add_pr_curve_tensorboard(i, test_probs, test_preds)

In [ ]:
%tensorboard --logdir logs